In [5]:
! pip install -e ..

Obtaining file:///Users/s2341012/Library/CloudStorage/Dropbox/DISMaL_chapter/DISMaL
  Preparing metadata (setup.py) ... done
  Attempting uninstall: dismal
    Found existing installation: dismal 0.1
    Uninstalling dismal-0.1:
      Successfully uninstalled dismal-0.1
  Running setup.py develop for dismal


In [6]:
from dismal.likelihood_matrix import LikelihoodMatrix
from dismal.demography import Epoch
import numpy as np
import matplotlib.pyplot as plt
from dismal.markov_matrices import TransitionRateMatrix, StochasticMatrix

In [7]:
S = np.array([
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1]
])

In [8]:
epoch0 = Epoch(id=0, allow_migration=False)
epoch1 = Epoch(id=1, allow_migration=True, symmetric_migration=False, migration_direction=None)
epoch2 = Epoch(id=2, allow_migration=True)

In [14]:
LikelihoodMatrix(params=[1,1,1,1,1,1,1,0.5,0.5,0.5,0.5], S=np.ones(shape=(3,10)), epoch_objects=[epoch0, epoch1, epoch2]).matrix

array([[0.84242651, 1.60213032, 2.31050187, 3.00141071, 3.69062159,
        4.38159278, 5.07389379, 5.76677652, 6.45985363, 7.15298459],
       [0.84242651, 1.60213032, 2.31050187, 3.00141071, 3.69062159,
        4.38159278, 5.07389379, 5.76677652, 6.45985363, 7.15298459],
       [1.61515115, 1.48896043, 1.78984599, 2.29930387, 2.91485817,
        3.57928333, 4.26319223, 4.95374699, 5.64625299, 6.33925875]])

In [56]:
from scipy.stats import poisson

def poisson_cdf(s, time, rate_param):

    if time is None:
        return 0
    else:
        return poisson.cdf(k=s, mu=(time*(rate_param)))
    

def diagonal_matrix_mult(mat, mat_inv):
    return -mat_inv @ np.diag(mat[:, 3])

def eigenvalue_matrix(eigenvals, start_time, end_time, rel_mu=1):
    """Generate matrix of s values (columns) x adjusted eigenvalues (rows).
    This corresponds to P(s events before coalescence) * 1-P(s events)"""

    if len(eigenvals) > 1:   
        lmbda = -np.array(eigenvals)[0:3]
    else:
        lmbda = -np.array(eigenvals)[0]
        
    s_vals = [s for s in range(0, S.shape[1])]

    eigv_mat = [(lmbda/(lmbda+rel_mu))
                * ((rel_mu/(lmbda+rel_mu))**s)
                * np.exp(lmbda*start_time)
                * poisson_cdf(s, time=start_time, rate_param=(lmbda+rel_mu))
                - poisson_cdf(s, time=end_time, rate_param=(lmbda+rel_mu))
                for s in s_vals]
    return np.transpose(np.array(eigv_mat))

In [74]:
Q1 = TransitionRateMatrix(thetas=[1,1], Ms=[0,0])
P = StochasticMatrix(Q1.eigenvectors, Q1.eigenvectors_inv, Q1.eigenvalues, t=1)
A = eigenvalue_matrix(Q1.eigenvalues, 0, 1)
B = eigenvalue_matrix(Q1.eigenvalues, 1, 2)
Gamma = eigenvalue_matrix([-1], 2, None)

In [75]:
A

array([[ 0.36466472, -0.15600585, -0.55167642, -0.79462346, -0.91609698,
        -0.96781139, -0.98765369, -0.99499703, -0.99780943, -0.99897694],
       [ 0.36466472, -0.15600585, -0.55167642, -0.79462346, -0.91609698,
        -0.96781139, -0.98765369, -0.99499703, -0.99780943, -0.99897694],
       [-0.36787944, -0.73575888, -0.9196986 , -0.98101184, -0.99634015,
        -0.99940582, -0.99991676, -0.99998975, -0.99999887, -0.99999989]])

In [79]:
(
    (-Q1.eigenvectors_inv @ np.diag(Q1.eigenvectors[:, 3]))[0, 0:3]@A 
    + P.matrix[0, 0:3]@(-Q1.eigenvectors_inv @ np.diag(Q1.eigenvectors[:, 3]))[0:3, 0:3]@B
    +(1-(P.matrix@P.matrix)[0,3])*Gamma
    )

array([0.85241874, 0.81982051, 0.64371168, 0.42290384, 0.24667222,
       0.13218098, 0.06603479, 0.03100874, 0.01382532, 0.00594267])

In [52]:
LikelihoodMatrix(params=[1,1,1,1,1,1,1,0,0,0,0], S=S, epoch_objects=[epoch0, epoch1, epoch2])

[[[0.78257772 1.68287544 2.63760526 3.55304993 4.37912461 5.13081962
   5.8442701  6.54329475 7.23791671 7.93139191]
  [0.78257772 1.68287544 2.63760526 3.55304993 4.37912461 5.13081962
   5.8442701  6.54329475 7.23791671 7.93139191]
  [3.69314718 3.28768207 3.47000363 3.92676203 4.51982575 5.1755854
   5.85657438 6.54627477 7.2385621  7.9315183 ]]

 [[0.78257772 1.68287544 2.63760526 3.55304993 4.37912461 5.13081962
   5.8442701  6.54329475 7.23791671 7.93139191]
  [0.78257772 1.68287544 2.63760526 3.55304993 4.37912461 5.13081962
   5.8442701  6.54329475 7.23791671 7.93139191]
  [3.69314718 3.28768207 3.47000363 3.92676203 4.51982575 5.1755854
   5.85657438 6.54627477 7.2385621  7.9315183 ]]

 [[0.48406673 0.85705974 1.30807449 1.86724583 2.49958544 3.16940848
   3.85490029 4.54587094 5.23847473 5.9315012 ]
  [0.48406673 0.85705974 1.30807449 1.86724583 2.49958544 3.16940848
   3.85490029 4.54587094 5.23847473 5.9315012 ]
  [1.69314718 1.28768207 1.47000363 1.92676203 2.51982575 3.17

In [5]:
from dismal.markov_matrices import TransitionRateMatrix, StochasticMatrix
Q = TransitionRateMatrix(thetas=[1,1], Ms=[0.5,0.4])
P = StochasticMatrix(Q.eigenvectors, Q.eigenvectors_inv, Q.eigenvalues, t=1)
Qs = [Q]
Ps = [P]
taus = [1]
thetas = [1]

In [6]:
epoch1.n_M_params

2

In [9]:
lm = LikelihoodMatrix([1,1,1,1,1,0.5,0.5], S, [epoch0, epoch1])

In [10]:
lm

[[0.77274718 1.39398483 1.98006919 2.59936487 3.25571183]
 [0.77274718 1.39398483 1.98006919 2.59936487 3.25571183]
 [1.44704322 1.27220353 1.54885456 2.04679072 2.65728213]]

In [48]:
poisson.cdf(k=1,mu=())

1.0

In [50]:
lmbda = 1
rel_mu = 1
s = 5
start_time, end_time = 1,2
((lmbda/(lmbda+rel_mu)) * ((rel_mu/(lmbda+rel_mu))**s) 
 * np.exp(lmbda*start_time) 
 * poisson_cdf(s, time=start_time, rate_param=(lmbda+rel_mu)) 
 - poisson_cdf(s, time=end_time, rate_param=(lmbda+rel_mu)))


-0.7433607421473977

In [63]:
poisson_cdf(s=10, time=10, rate_param=1)

0.5830397501929852

In [36]:
i = 0 
(np.identity(4)[i, 0:3]
 @ diagonal_matrix_mult(Qs[0].eigenvectors, Qs[0].eigenvectors_inv)[0:3, 0:3]
@ eigenvalue_matrix(eigenvals=Qs[0].eigenvalues, start_time=0, end_time=taus[0])) 
                                 #+ (1 - (np.identity(4)@Ps[0].matrix)[i, 3]))
                                 #* eigenvalue_matrix(eigenvals=[-1/thetas[0]],
                                  #                         start_time=taus[0], end_time=None))


[1.63277244 1.43832013 0.27890743]


array([ 0.25578454, -0.23624409, -0.56550895, -0.76722586, -0.87952267])

In [37]:
eigenvalue_matrix([-1, -1, 0], start_time=1, end_time=2)

[1 1 0]


array([[ 0.16562408,  0.18433139, -0.00817865, -0.28785117, -0.54836331],
       [ 0.16562408,  0.18433139, -0.00817865, -0.28785117, -0.54836331],
       [-0.13533528, -0.40600585, -0.67667642, -0.85712346, -0.94734698]])

In [68]:
lmbda = 1
rel_mu = 1
s = 3
start_time = 1
end_time = 2

((lmbda/(lmbda+rel_mu)) * ((rel_mu/(lmbda+rel_mu))**s) * np.exp(lmbda*start_time)
* poisson_cdf(s, time=start_time, rate_param=(lmbda+rel_mu))
- poisson_cdf(s, time=end_time, rate_param=(lmbda+rel_mu)))

-0.28785117490301304

In [69]:
poisson_cdf(s, time=start_time, rate_param=(lmbda+rel_mu))

0.857123460498547

In [71]:
poisson_cdf(s, time=end_time, rate_param=(lmbda+rel_mu))

0.43347012036670896

In [72]:
(lmbda/(lmbda+rel_mu)) * ((rel_mu/(lmbda+rel_mu))**s) * np.exp(lmbda*start_time)

0.16989261427869032

In [16]:
eigenvalue_matrix(eigenvals=[-1/thetas[0]], start_time=taus[0], end_time=None)

array([0.18393972, 0.27590958, 0.22992465, 0.14561895, 0.08047363])

In [65]:
poisson_cdf(s=1, time=None, rate_param=(-1/thetas[0]+1))

0

In [33]:
np.identity(4)@Ps[0].matrix

array([[0.36787944, 0.        , 0.        , 0.63212056],
       [0.        , 0.36787944, 0.        , 0.63212056],
       [0.        , 0.        , 1.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        ]])

In [29]:
np.identity(4)@Ps[0]#[i, 3]

ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

In [28]:
np.identity(4)[i, 0:3] @ self.diagonal_matrix_mult(self.Qs[0].eigenvectors, self.Qs[0].eigenvectors_inv)[0:3, 0:3]
                  @ self.eigenvalue_matrix(eigenvals=self.Qs[0].eigenvalues,
                                           start_time=0, end_time=self.taus[0])
                                           + (1 - (np.identity(4)@self.Ps[0])[i, 3])
                                           * self.eigenvalue_matrix(eigenvals=[1/self.thetas[0]]*3,
                                                                    start_time=self.taus[0], end_time=None)

IndentationError: unexpected indent (3852645994.py, line 2)

In [26]:
params = [1,1,1,1,1,1,1,0,0,0,0]
S = np.ones(shape=(3, 10))
epochs = [
    Epoch(id=0, allow_migration=False),
    Epoch(id=1, allow_migration=True),
    Epoch(id=2, allow_migration=True)
]

lm = LikelihoodMatrix(params, S, epochs)

taus = lm.taus
Qs = lm.Qs
Ps = lm.Ps

In [45]:
i = 2
(diagonal_matrix_mult(Qs[0].eigenvectors, Qs[0].eigenvectors_inv)[0:3, 0:3]
            @ eigenvalue_matrix(eigenvals=Qs[0].eigenvalues, start_time=0, end_time=taus[1])
            + Ps[0][i, 0:3]
            @ diagonal_matrix_mult(Qs[1].eigenvectors, Qs[1].eigenvectors_inv)[0:3, 0:3]
            @ eigenvalue_matrix(eigenvals=Qs[1].eigenvalues, start_time=taus[1], end_time=taus[0])
            + (1 - (Ps[0].matrix@Ps[1].matrix)[i, 3])
            * eigenvalue_matrix(eigenvals=[-1/thetas[0]], start_time=taus[0], end_time=None))

array([[0.91648388, 0.85566261, 0.59794684, 0.33200733, 0.1607168 ,
        0.07336407, 0.03340336, 0.01559936, 0.00749733, 0.0036774 ],
       [0.91648388, 0.85566261, 0.59794684, 0.33200733, 0.1607168 ,
        0.07336407, 0.03340336, 0.01559936, 0.00749733, 0.0036774 ],
       [0.18393972, 0.27590958, 0.22992465, 0.14561895, 0.08047363,
        0.04176964, 0.02114029, 0.01060664, 0.00530788, 0.00265445]])

In [48]:
diagonal_matrix_mult(Qs[0].eigenvectors, Qs[0].eigenvectors_inv)[0:3, 0:3] @ eigenvalue_matrix(eigenvals=Qs[0].eigenvalues, start_time=0, end_time=taus[1])

array([[0.73254416, 0.57975303, 0.36802219, 0.18638838, 0.08024317,
        0.03159442, 0.01226306, 0.00499272, 0.00218945, 0.00102295],
       [0.73254416, 0.57975303, 0.36802219, 0.18638838, 0.08024317,
        0.03159442, 0.01226306, 0.00499272, 0.00218945, 0.00102295],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]])

In [46]:
Ps[0][i, 0:3] @ diagonal_matrix_mult(Qs[1].eigenvectors, Qs[1].eigenvectors_inv)[0:3, 0:3] @ eigenvalue_matrix(eigenvals=Qs[1].eigenvalues, start_time=taus[1], end_time=taus[0])

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [39]:
eigenvalue_matrix(eigenvals=Qs[0].eigenvalues, start_time=0, end_time=taus[1])

array([[ 0.36466472, -0.15600585, -0.55167642, -0.79462346, -0.91609698,
        -0.96781139, -0.98765369, -0.99499703, -0.99780943, -0.99897694],
       [ 0.36466472, -0.15600585, -0.55167642, -0.79462346, -0.91609698,
        -0.96781139, -0.98765369, -0.99499703, -0.99780943, -0.99897694],
       [-0.36787944, -0.73575888, -0.9196986 , -0.98101184, -0.99634015,
        -0.99940582, -0.99991676, -0.99998975, -0.99999887, -0.99999989]])